In [1]:
%load_ext autoreload
%autoreload 2

# Init

In [2]:
import sys
import os
sys.path.insert(0, "../src")
sys.path.append("/users/k21190024/study/fact-checking-repos/fever/teamathene/src")

import json
import random
import pickle as pkl
import numpy as np
import scipy
from pathlib import Path
from collections import Counter, defaultdict
from joblib import Parallel, delayed

import constants
from evaluate import score
from gen.util import read_data, write_jsonl

# Inspect Data class

In [3]:
from athene.retrieval.sentences.data_processing.data import Data
from athene.utils.config import Config
from scripts.athene import pipeline

os.chdir("/users/k21190024/study/fact-checking-repos/fever/teamathene")
Config.load_config("/users/k21190024/study/fact-checking-repos/fever/teamathene/conf/fever_config.json")

Deprecated cache directory found (/users/k21190024/.allennlp/datasets).  Please remove this directory from your system to free up space.
/scratch/users/k21190024/envs/conda/fever-teamathene/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/users/k21190024/envs/conda/fever-teamathene/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/users/k21190024/envs/conda/fever-teamathene/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
args = pipeline._construct_args_for_sentence_retrieval(phase="data")

data = Data(args.embedding_path, args.train_data, args.dev_data, args.test_data, args.fasttext_path,
                num_negatives=args.num_negatives, h_max_length=args.c_max_length, s_max_length=args.s_max_length,
                random_seed=args.random_seed, reserve_embed=args.reserve_embed, db_filepath=args.db_path)

./data/cfever-embedding/test_data.p
./data/fasttext/wiki.en.bin
Embedding size: 10644
embed_dict size 10644
Generate numpy embed: (10645, 300)


In [10]:
X_train = data.train_data_loader(os.path.join(data.embedding_path, "train_sample.p"), data.train_file, num_samples=data.num_negatives)
f_train = read_data(Path("/users/k21190024/study/fact-checking-repos/fever/teamathene/data/fever-data/train.jsonl"))

In [11]:
X_train_claims = set([i[0] for i in X_train])
f_train_claims = set([i["claim"] for i in f_train])
train_included = [i["claim"] in X_train_claims for i in f_train]
tr_in = [j for i, j in zip(train_included, f_train) if i]
tr_out = [j for i, j in zip(train_included, f_train) if not i]

In [12]:
len(tr_in), len(tr_out)

(0, 145449)

In [13]:
Counter([i["label"] for i in tr_in]), Counter([i["label"] for i in tr_out])

(Counter(),
 Counter({'SUPPORTS': 80035, 'REFUTES': 29775, 'NOT ENOUGH INFO': 35639}))